In [244]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
# import google.generativeai as genai
from google import genai
import time
import ast
from pydantic import ValidationError
from pydantic import BaseModel
from typing import Optional
import enum
import pprint

In [211]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
# genai.configure(api_key=GEMINI_API_KEY)

In [168]:
filtrados_true = pd.read_excel('../docs/resposta_danos_ambientais_totalamostra.xlsx')
filtrados_true = filtrados_true.loc[filtrados_true['processo_ambiental'] != False]
filtrados_true.head(5)

,Unnamed: 0.1,Unnamed: 0,processo,processo_ambiental,justificativa,processoAnexoID,link de referencia
1,1,0000135-52.2021.8.16.0067,0000135-52.2021.8.16.0067,True,Trata-se de Ação Civil Pública ajuizada pelo M...,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
3,3,0000272-92.1992.8.26.0157,0000272-92.1992.8.26.0157,True,Ação Civil Pública - Meio Ambiente,657568057,http://jud-anexos.digesto.com.br/4f696a2b03b85...
4,4,0000428-83.2019.8.16.0134,0000428-83.2019.8.16.0134,True,Assunto Principal: Dano Ambiental,1088560478,http://jud-anexos.digesto.com.br/05d31619ede92...
5,5,0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,True,"Na petição inicial, dentre seus pedidos, os au...",1186043580,http://jud-anexos.digesto.com.br/7487b4175300b...
7,7,0000640-15.2017.8.11.0102,0000640-15.2017.8.11.0102,True,O Ministério Público do Estado de Mato Grosso ...,1299854564,http://jud-anexos.digesto.com.br/40b55644f0bfc...


In [169]:
filtrados_com_num = pd.read_excel("../docs/processos_filtrados.xlsx")
filtrados_com_num.head(10)

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
5,5,5003677-41.2019.8.13.0521,612465137,590939497,http://jud-anexos.digesto.com.br/51644135dd7d3...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
6,6,0000527-75.2011.8.02.0020,582564055,1186042345,http://jud-anexos.digesto.com.br/b9775f943d51d...,https://www2.tjal.jus.br/pastadigital/getPDF.d...,LAUDO,2016-08-17,2024-09-12T04:13:58.579221,TJAL,LAUDO DE AVALIACAO - PAGINA 99,0000527-75.2011.8.02.0020
7,7,0011297-52.2011.8.13.0140,690025576,2589045924,http://jud-anexos.digesto.com.br/6a7cee8b71f2a...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO PERICIAL - PARTE 01 - LAUDO PERICIAL CAR...,0011297-52.2011.8.13.0140
8,8,0011297-52.2011.8.13.0140,690025576,2589045929,http://jud-anexos.digesto.com.br/0584da20569c9...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO DE PERICIA - PARTE 02 - LAUDO OFICIAL CA...,0011297-52.2011.8.13.0140
9,9,0011297-52.2011.8.13.0140,690025576,2589045772,http://jud-anexos.digesto.com.br/9d774ad2a90a2...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2024-01-25,2025-02-21T07:26:39.772134,TJMG,LAUDO - LAUDO COMPLEMENTAR CARMENSE COMERCIAL ...,0011297-52.2011.8.13.0140


In [170]:
processos_unicos = list(filtrados_com_num['num_processo'].unique())
display(filtrados_com_num.loc[filtrados_com_num['num_processo']=='Não encontrado'])
processos_unicos.remove('Não encontrado')

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
51,51,5580808-19.2022.8.09.0011,503987107,972210470,http://jud-anexos.digesto.com.br/313133a43d0b9...,https://projudi.tjgo.jus.br/BuscaProcesso?Pagi...,DECISAO,2023-01-18,2024-08-23T05:34:28.806878,TJGO,DECISAO DE PEDIDO DE URGENCIA,Não encontrado
55,55,5061164-56.2024.8.09.0051,570015135,2564299827,http://jud-anexos.digesto.com.br/2a8bb47194987...,https://projudi-2024-prd.s3.tjgo.jus.br/202412...,DECISAO,2024-12-04,2025-02-19T07:58:29.294476,TJGO,DECISAO,Não encontrado
102,102,0000257-34.2024.8.05.0110,655333385,1942855138,http://jud-anexos.digesto.com.br/30f8d4e93b94a...,https://projudi.tjba.jus.br/projudi/listagens/...,DOCUMENTO,2024-02-26,2024-11-30T22:32:58.025278,TJBA,DOCUMENTO,Não encontrado


In [171]:
filtrados_com_num.head()

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521


In [238]:
class FormatoResposta(BaseModel):
    georreferencia: str
    uf: str
    municipio: str
    responsavel: str
    categoria_responsavel: str
    tipo_impacto: str
    descricao_impacto: str
    data_impacto: str
    area_afetada: str
    unidade_area: str
    houve_compensacao: bool
    categoria_compensacao: str
    tipo_multa: int | str
    valor_multa: float | str
    valor_multa_diaria: float | str

In [239]:
def analise(texto_extraido):
    prompt = f"""
        SYSTEM: Você é meu assistente especialista em análise e extração de elementos de textos judiciais. Você irá realizar extrações especificamente sobre
        danos socioambientais de diversos tipos, pensando em futuramente usar a tabela gerada para fazer uma modelagem preditiva de multas para danos socioambientais.

        INSTRUCTIONS:
        Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
        Seu objetivo é analisar esse texto e extrair 15 informações específicas para montar um banco de dados, conforme as instruções abaixo.
        Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
        de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
        Retorno esperado:

        [0] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
         Se não houver, retorne: NULL

        [1] Sigla da Unidade Federativa (UF) (ex: "SP", "MG" etc.)
         Se não houver, retorne: NULL

        [2] Município ou cidade do local afetado. (ex: "São Paulo", "Minas Gerais" etc)
         Se não houver, retorne: NULL

        [3] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
         Se não houver, retorne: NULL

        [4] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
         Se não houver, retorne: NULL

        [5] Tipo de impacto: Categoria do dano (ex.: "Desmatamento de APP", "Derramamento de Petróleo", "Poluição Hídrica").
        Se não houver, retorne: NULL

        [6] Descrição do impacto: Resuma o impacto em até 30 palavras.
        Se não houver, retorne: NULL

        [7] Data do impacto ambiental no formato: DD/MM/AA
         Se não houver, retorne: NULL

        [8] Extensão da área afetada (ex: "15000".)
         Se não houver, retorne: NULL
        Sempre em metros quadrados (m²) ou hectares (ha), então converta se necessário. Não use separadores para milhares

        [9] Unidade de medida da área (ex: "ha" ou "m2")
         Se não houver, retorne: NULL

        [10] Houve compensação não monetária atribuída (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
         Se houver, retorne: "True"
         Se não houver, retorne: "False"

        [11] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", "Custas Judiciais e Acordos" ou "Valoração Econômica".
         Se não houver, retorne: NULL
         Tente não fugir dessas categorias.

        [12] Tipo de multa: (0, 1 ou 2)
        Se for uma única multa aplicada sobre o responsável, coloque 0
        Se for algo como uma multa diária, coloque 1.
        Se forem os dois, como uma multa imediata e um pagamento diário, coloque 2

        [13] Valor da multa completa para o Tipo 0 ou 2: (ex: "123000.00", "999999.99")
        Caso o item 12 seja 0, complete essa coluna com o valor total da multa aplicada.
        Caso o item 12 seja 1, deixe como NULL
        Caso o item 12 seja 2, complete essa coluna com o valor total da multa imediata aplicada.

        [14] Valor da multa diária para o Tipo 1 ou 2: (ex: "123000.00", "10000.50")
        Caso o item 12 seja 1 ou 2, complete essa coluna com o valor diário a ser pago pelo responsável.
        Caso o item 12 seja 0, deixe como NULL

        IMPORTANTE: A string será usada para um dataframe posteriormente. Logo:
        NÃO inclua explicações, formatações extras, aspas ou colchetes na resposta.
        NÃO deixe nenhuma casa em branco ou vazia. SEMPRE coloque NULL caso não encontre no texto.
        NÃO inclua o texto original, nem explicações, apenas a string limpa.
        A ordem dos elementos PRECISA SER RESPEITADA!! NENHUM ELEMENTO PODE VIR ANTES DA SUA POSIÇÃO!!

        USER:
        Texto para análise:
        {texto_extraido}
    """
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta
            # 'max_output_tokens': 500,
            # 'temperature': 1.0
        }
    )
    time.sleep(3)
    return resposta

In [234]:
processos_filtrados_com_num = pd.read_excel("../docs/processos_filtrados_com_num.xlsx")
processos_filtrados_com_num.head()

,Unnamed: 0,processoID,processoAnexoID,Download copia,Tribunal,num_processo
0,0,474474022,2344968075,http://jud-anexos.digesto.com.br/5ab34519c8f96...,TJAM,0600357-89.2021.8.04.5600
1,1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600
2,2,474474806,2294638419,http://jud-anexos.digesto.com.br/2adc94fb608d8...,TJAM,0600384-72.2021.8.04.5600
3,6,474474806,2294638453,http://jud-anexos.digesto.com.br/7abb2bb3a616f...,TJAM,0600384-72.2021.8.04.5600
4,7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600


In [240]:
respostas = {}
lista_numeros = list(filtrados_true['processo'].unique())

camila = lista_numeros[0:12]
rafa = lista_numeros[12:25]
nasser = lista_numeros[25:37]
nery = lista_numeros[37:50]

nery

['0900008-39.2019.8.12.0013',
 '0900394-32.2018.8.24.0058',
 '1000009-75.2018.4.01.3903',
 '1000015-88.2019.8.11.0038',
 '1000061-60.2024.8.11.0084',
 '1000098-84.2022.8.26.0587',
 '1000098-88.2019.4.01.3604',
 '1000228-29.2019.8.11.0092',
 '1000305-06.2018.4.01.3901',
 '1000415-46.2020.4.01.3800',
 '1000589-43.2019.8.11.0093',
 '1000802-59.2021.8.26.0126',
 '1000908-82.2022.8.11.0100']

In [249]:
for processo in nery[0:1]: 
    linhas_correspondentes = processos_filtrados_com_num.loc[processos_filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        processoAnexoID = row['processoAnexoID']
        link = row['Download copia']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()

        resposta_prompt = analise(texto)
        print(resposta_prompt.text)
        # resposta_prompt = dict(resposta_prompt.text)
        # resposta_prompt["numero_processo"] = processo
        # resposta_prompt["processoAnexoID"] = processoAnexoID
        # resposta_prompt["link_referencia"] = link
        # respostas[processo] = resposta_prompt
        # pprint.pprint(f"Processo: {processo} - Resposta: {respostas}")
        break

{
"georreferencia": "NULL",
"uf": "MS",
"municipio": "Guia Lopes da Laguna",
"responsavel": "Agência Estadual de Gestão de Empreendimentos - AGESUL",
"categoria_responsavel": "Pessoa Jurídica",
"tipo_impacto": "Deterioração de rio por escombros",
"descricao_impacto": "Queda de ponte de concreto causa obstrução do leito do rio, assoreamento, prejuízo à reprodução dos peixes e impacto socioeconômico na população ribeirinha.",
"data_impacto": "NULL",
"area_afetada": "NULL",
"unidade_area": "NULL",
"houve_compensacao":  true,
"categoria_compensacao": "Obrigações de Fazer (com custo)",
"tipo_multa": 1,
"valor_multa": "NULL",
"valor_multa_diaria": "1000.00"
}


In [148]:
respostas

{'0600592-22.2022.8.04.5600': ['0600592-22.2022.8.04.5600',
  'NULL',
  'AM',
  'Manicoré',
  'C R MADEIREIRA AMAZONAS LTDA-EPP',
  'Pessoa Jurídica',
  'Queima de rejeitos de indústria madeireira',
  'Lançamento e queima de rejeitos de indústria madeireira a céu aberto.',
  'NULL',
  'NULL',
  'NULL',
  'False',
  'Multas Administrativas',
  '0',
  '50000.00',
  'NULL',
  1915312980,
  'http://jud-anexos.digesto.com.br/4dee720275cd555776cfe8fec5a8f850.pdf'],
 '0643812-33.2015.8.04.0001': ['0643812-33.2015.8.04.0001',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  29931630,
  'http://jud-anexos.digesto.com.br/632dc985404a0b5c59d6ab8997d9b05f.pdf'],
 '0800007-14.2021.8.01.0003': ['0800007-14.2021.8.01.0003',
  'NULL',
  'AC',
  'Brasiléia',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  'NULL',
  2178216543,
  'http://jud-anexos.digesto.com.br/bf9d0dbe0b966977b1554ecbe

In [149]:
respostas_teste1 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste1.columns = [
    "processo",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "topo_impacto",
    "descricao_impacto",
    "data",
    "area",
    "unidade",
    'compensacao_nao_monetaria',
    "categoria_compensacao",
    "tipo_multa",
    "valor_multa",
    "valor_multa_diaria",
    "id_documento",
    "url_documento"
]

respostas_teste1 = respostas_teste1.reset_index(drop=True)
respostas_teste1

,processo,georreferência,UF,municipio,responsavel,tipo_responsavel,topo_impacto,descricao_impacto,data,area,unidade,compensacao_nao_monetaria,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,id_documento,url_documento
0,0600592-22.2022.8.04.5600,NULL,AM,Manicoré,C R MADEIREIRA AMAZONAS LTDA-EPP,Pessoa Jurídica,Queima de rejeitos de indústria madeireira,Lançamento e queima de rejeitos de indústria m...,NULL,NULL,NULL,False,Multas Administrativas,0,50000.00,NULL,1.915313e+09,http://jud-anexos.digesto.com.br/4dee720275cd5...
1,0643812-33.2015.8.04.0001,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,29931630,http://jud-anexos.digesto.com.br/632dc985404a0...,NaN,None
2,0800007-14.2021.8.01.0003,NULL,AC,Brasiléia,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2178216543,http://jud-anexos.digesto.com.br/bf9d0dbe0b966...,None,NaN,None
3,0800450-30.2021.8.14.0123,NULL,PA,Novo Repartimento,Madeireira Nordestina Ltda.,Pessoa Jurídica,Inserção de informações falsas em sistema de c...,Inserção fraudulenta de informações no SISFLOR...,NULL,NULL,NULL,False,Multas Administrativas,0,456500.00,NULL,1.470467e+09,http://jud-anexos.digesto.com.br/ada399d8c52be...
4,0800467-03.2014.8.12.0015,NULL,MS,Miranda,Orlando Alvarenga Pinto Coelho,Pessoa Física,Falta de Reserva Legal,"Propriedade rural sem reserva legal averbada, ...",NULL,NULL,NULL,False,NULL,1,NULL,1000.00,2.461927e+09,http://jud-anexos.digesto.com.br/b40d332969a6a...
5,0800469-50.2022.8.14.0107,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,266667425,http://jud-anexos.digesto.com.br/5c3edc578abed...,None,NaN,None
6,0800550-82.2021.8.14.0123,NULL,PA,Novo Repartimento,Sarandí Madeiras Maracajá,Pessoa Jurídica,Exploração ilegal de madeira,Inserção de informações falsas no SISFLORA/PA ...,NULL,19403,m2,False,Multas Administrativas,0,311500.00,NULL,1.404986e+09,http://jud-anexos.digesto.com.br/5cad1eab93ad3...
7,0800572-43.2021.8.14.0123,NULL,PA,Novo Repartimento,Madeireira Nordestina Ltda.,Pessoa Jurídica,Exploração ilegal de madeira,Inserção de informações falsas em sistemas ofi...,NULL,NULL,NULL,False,Multas Administrativas,0,411500.00,NULL,1.965108e+09,http://jud-anexos.digesto.com.br/32a8fd7dc2205...
8,0800665-06.2021.8.14.0123,NULL,PA,Novo Repartimento,INDUSTRIA E COMERCIO DE MADEIRAS JUREMA LTDA,Pessoa Jurídica,Exploração ilegal de madeira,Venda de madeira serrada sem guias florestais.,NULL,NULL,NULL,False,Multas Administrativas,0,6300.00,NULL,1.347196e+09,http://jud-anexos.digesto.com.br/4df0ebd528fe0...
9,0800773-25.2013.8.12.0041,NULL,MS,Ribas do Rio Pardo,Donino Carlos Amorim,Pessoa Física,Assoreamento e degradação de Área de Preservaç...,Acesso de gado em APP causando erosão e assore...,16/12/04,NULL,NULL,True,Obrigações de Fazer (com custo),0,NULL,NULL,2.389735e+09,http://jud-anexos.digesto.com.br/c1d25a940c174...
